In [1]:
# %load std_libraries.py
import sys
#Indicate operating environment and import core modules
location_input = input("what computer are you on? a = Bens, b = gpucluster, c = other   ")
location_dict = {'a': "C:\\Users\\heine\\github\\expression_broad_data", 'b': "/home/heineike/github/expression_broad_data",'c':'you need to add your location to the location_dict'}
base_dir = location_dict[location_input]
print("base directory is " + base_dir)

if sys.path[-1] != base_dir:
    sys.path.append(base_dir)
    print("Added " + base_dir + " to path: " )
    print(sys.path)

import os

print("I am about to import a library")
from core import expression_plots 
from core import io_library 
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook
data_processing_dir = base_dir + os.sep + os.path.normpath("expression_data") + os.sep

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns; sns.set(style="ticks", color_codes=True)
#from sklearn import linear_model
#import pickle
#import subprocess
#import networkx as nx
# import scipy.stats as stats
import scipy.cluster.hierarchy as sch

# from Bio import SeqIO
# from Bio import SeqFeature as sf
# from Bio.Alphabet import generic_dna
# from Bio.Seq import Seq

# import re

from collections import Counter
# import scipy.stats as stats
# from itertools import chain
#this only works if you are online
online_input = input("are you online? Yes/No")
if online_input == "Yes": 
    import plotly.plotly as py
    import plotly.graph_objs as pygo
    import plotly.tools as pytools
    py.sign_in('heineike02_student','9dMTMZgJMgUP0YX0P5mQ')
    #py.sign_in('heineike02', 'APPjKrtARaN2ZgUYIkqr')


what computer are you on? a = Bens, b = gpucluster, c = other   b
base directory is /home/heineike/github/expression_broad_data
Added /home/heineike/github/expression_broad_data to path: 
['', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/usr/local/lib/python3.5/dist-packages', '/usr/local/lib/python3.5/dist-packages/cmonkey2-1.2.6-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/SQLAlchemy_Utils-0.32.14-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/svgwrite-1.1.11-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/Routes-2.4.1-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/CherryPy-10.2.2-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/biopython-1.69-py3.5-linux-x86_64.egg', '/usr/local/lib/python3.5/dist-packages/configparser-3.5.0-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/rpy2-2.8.6-py3.5-linux-x86_64.egg', '/usr/local/lib/python3.5/dist-packages/repoze.lru-0.6-py3.5.egg', '

## Load Data

In [2]:
#Read in the KL promoter database.  
kl_promoters = pd.read_pickle(data_processing_dir + os.path.normpath("kl_promoters/kl_promoters.pkl"))

#Read in the SC promoter database.  
sc_promoters = pd.read_pickle(data_processing_dir + os.path.normpath("sc_promoters/sc_promoters.pkl"))

In [3]:
# Load goi data from goi data file created in paralog analysis script. 
goi_data = pd.read_csv(data_processing_dir + os.sep + "20170817_klscpka" + os.sep + "goi_data_solis.csv")

## Build enrichment sets

Did enrichment analysis using DREME, MEME, and GLAM2 on the upregulated set.  

Only found Msn2 and a few others 


In [21]:
a = '2.0'

In [27]:
#Upregulated v.s. all
LFC_cutoff_high = 2.0
genes = goi_data[goi_data['log2FoldChange_SC']>LFC_cutoff_high]['sc_genename']

target_promoter_fname = data_processing_dir + os.path.normpath('sc_promoters/promoter_sets/paralogs_LFC_gt_' + str(LFC_cutoff_high).replace('.','p') + '.fasta')

io_library.write_promoter_file(sc_promoters, list(genes),target_promoter_fname)

#Downregulated v.s. all
LFC_cutoff_low = -2.0
genes = goi_data[goi_data['log2FoldChange_SC']<LFC_cutoff_high]['sc_genename']

target_promoter_fname = data_processing_dir + os.path.normpath('sc_promoters/promoter_sets/paralogs_LFC_lt_' + str(LFC_cutoff_low).replace('-','m').replace('.','p') + '.fasta')

io_library.write_promoter_file(sc_promoters, list(genes),target_promoter_fname)



#Upregulated v.s. flat and downregulated
#downregulated v.s. flat and upregulated

all_sc_promoter_fname = data_processing_dir + os.path.normpath('sc_promoters/promoter_sets/all_sc_promoters.fasta')

YIL111W not in promoter data set.


## Run MEME to look for unbiased enrichment

In [ ]:
#Right now meme doesn't do discriminatory and allow any number of motifs to be scanned. 
#Dreme seems to be better for that. 
#Ran GLAM2 and got a bunch of polyA sites. 


#run psp-gen to get prior for positive and negative sequences. 
psp_gen_command = ['psp-gen', '-pos', target_promoter_fname, '-neg', all_sc_promoter_fname, '-dna', 
                   '-minw', '6', #minimum width of motif
                   '-maxw', '20', #maximum width of motif
                   '>', 'psp_prior.psp']
subprocess.run(psp_gen_command)

In [ ]:
meme paralogs_LFC_gt_2p0.fasta -dna -oc . -nostatus -time 17358 -maxsize 60000 -mod anr -nmotifs 5 -minw 6 -maxw 50 -revcomp -psp priors.psp
meme paralogs_LFC_gt_2p0.fasta -dna -oc . -nostatus -time 17402 -maxsize 60000 -mod zoops -nmotifs 3 -minw 6 -maxw 50 -revcomp -psp priors.psp
dreme dreme -v 1 -oc . -dna -p paralogs_LFC_gt_2p0.fasta -n all_sc_promoters.fasta -t 18000 -e 0.05

In [ ]:
#Run FIMO for particular motifs
#Run AME with all JASPAR motifs
#Run exact motif matches

In [ ]:
io_library.SC_common_name_lookup([])